# Regular Representation of a Group or Monoid

## References

* Georgi, Howard (2018), <i>"Lie algebras in particle physics: from isospin to unified theories"</i>, CRC Press, [Open Access](https://www.taylorfrancis.com/books/oa-mono/10.1201/9780429499210/lie-algebras-particle-physics-howard-georgi?context=ubx&refId=1530fefc-3778-48ae-99ec-cba2935af2fb)
* Huang, Jiaqi (2012), <i>"Lie Groups and their applications to Particle Physics: A Tutorial for Undergraduate Physics Majors"</i>, [arXiv:2012.00834v1](https://arxiv.org/abs/2012.00834)

## Introduction

The following definition was adapted from the definition in [Georgi, 2018].

A <i>representation</i> of a group, $G = \langle A, \circ \rangle$, is a mapping, $V$, of the elements of $G$ onto a set of linear operators with the following properties:
* $V(e) = \hat{1}$, where $e$ is the group's identity element and $\hat{1}$ is the identity operator in the space on which the linear operators act
* $V(a_i) \cdot V(a_j) = V(a_i \circ a_j)$. That is, the group multiplication law (denoted by $\circ$) is mapped onto the natural multiplication in the linear space on which the linear operators act (denoted by $\cdot$).

The <i>regular representation</i> of a group is a mapping of each group element to an $nxn$ matrix. In this case, $\hat{1}$ is the $nxn$ identity matrix.

For example, the regular representation of the cyclic group, $Z_4$, with elements, $A = \left\{ R_0, R_{90}, R_{180}, R_{270} \right\}$, consists of the mapping shown below, as a Python dictionary.

In [1]:
import numpy as np  # Use NumPy to represent matrices & vectors

In [2]:
z4_reg_rep = {
    'R0': np.array([[1, 0, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]]),
    'R90': np.array([[0, 0, 0, 1],
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0]]),
    'R180': np.array([[0, 0, 1, 0],
                      [0, 0, 0, 1],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0]]),
    'R270': np.array([[0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1],
                      [1, 0, 0, 0]])
}

### Note on Monoids

The definition of a regular representation, given above, and the theory provided, below, doesn't preclude Monoids having regular represenations. The definition & theory require an algebra that has an identity element, but not inverses. Take away inverses from groups, and you have monoids. The discussion to follow will use groups as examples, but toward the end, some monoid examples will also be included.

## Theory

The following description of the algorithm for computing a regular representation was adapted from [Huang, 2012].

Let $G = \langle A, \circ \rangle$, be a group, where $A = \{a_0, a_1, \dots , a_{n - 1}\}$ is the set of the group's elements, and $\circ$ is its binary operator.

Also, let $B = \{\hat{b}_0, \hat{b}_1, \dots , \hat{b}_{n-1} \}$ be a set of $nx1$ orthogonal unit vectors:

$\hat{b}_0 = \begin{bmatrix}
1 \\
0 \\
0 \\
\vdots \\
0 \end{bmatrix},
\hat{b}_1 = \begin{bmatrix}
0 \\
1 \\
0 \\
\vdots \\
0 \end{bmatrix},
\dots,
\hat{b}_{n-1} = \begin{bmatrix}
0 \\
0 \\
0 \\
\vdots \\
1 \end{bmatrix}$

Define the a bijection between $A$ and $B$ as follows: $V(a_i) = \hat{b}_i$ for $i = 0, \dots , n - 1$.

Let $\cdot$ denote matrix-vector multiplication, and define the $nxn$ matrix,

$C_k = (c^k_{ij})_{i,j=0,\dots,n-1}$

where $c^k_{ij} = \hat{b}_i^T \cdot V(a_k \circ V^{-1}(\hat{b}_j))$



Then $M = \{C_0, C_1, \dots , C_{n - 1}\}$ is the <b>regular representation</b> of the group $G$, where the mapping between group elements and operators is $a_i \leftrightarrow C_i$ for $i = 0, \dots , n - 1$.

The purpose of the remainder of this notebook is to develop an implementation, within the module <b>finite_algebras</b>, that produces the mapping, $a_i \leftrightarrow C_i$, from a given group, $G$.

## A Small Test Group

In [3]:
import finite_algebras as alg

In [4]:
grp = alg.make_finite_algebra(
    'Z4',
    'Cyclic group of order 4',
    ['R0', 'R90', 'R180', 'R270'],
    [[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]
)

grp.about()  # Displays information about the group


** Group **
Name: Z4
Instance ID: 4611671824
Description: Cyclic group of order 4
Order: 4
Identity: R0
Commutative? Yes
Cyclic?: Yes
  Generators: ['R270', 'R90']
Elements:
   Index   Name   Inverse  Order
      0      R0      R0       1
      1     R90    R270       4
      2    R180    R180       2
      3    R270     R90       4
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]


In [5]:
a = grp.elements
n = grp.order

## The Bijection between Group Elements and Vectors

First create a dictionary that maps each of the group's elements to an n-dimensional orthogonal basis vector.

In [6]:
id = np.eye(n, dtype=int)  # The nxn identity matrix
b = [id[:,[i]] for i in range(n)]  # A list of the columns of the identity matrix
mapping = dict(zip(a, b))
mapping

{'R0': array([[1],
        [0],
        [0],
        [0]]),
 'R90': array([[0],
        [1],
        [0],
        [0]]),
 'R180': array([[0],
        [0],
        [1],
        [0]]),
 'R270': array([[0],
        [0],
        [0],
        [1]])}

The dictionary, above, maps group elements to "vectors" (NumPy arrays). To complete the bijection, we need another dictionary that goes in the reverse direction, "vectors" to group elements. However, dictionary keys must be immutable, and NumPy arrays are mutable, so the code, below, transforms NumPy arrays to tuples, which <i>are</i> immutable.

In [7]:
def to_tuple(vec):
    '''Turns a column vector into a tuple, for use as a dictionary key.'''
    return tuple(map(lambda x: x[0], list(vec)))

inv_mapping = {to_tuple(val): key for key, val in mapping.items()}
inv_mapping

{(1, 0, 0, 0): 'R0',
 (0, 1, 0, 0): 'R90',
 (0, 0, 1, 0): 'R180',
 (0, 0, 0, 1): 'R270'}

Now define the bijection. That is, $V$, and its inverse, $V^{-1}$.

In [8]:
def V(elem):
    '''Given a group element name, return the corresponding vector.'''
    return mapping[elem]

def Vinv(vec):
    '''Given a vector, return the corresponding element.'''
    return inv_mapping[to_tuple(vec)]

# Test/Example:
elem = a[1]
vec = V(elem)
elem2 = Vinv(vec)
print(elem)
print(vec)
print(elem2)

R90
[[0]
 [1]
 [0]
 [0]]
R90


## The Regular Representation Matrices

Now, the regular represention matrices can be derived using the formula presented in the Theory section, above,

$C_k = (c^k_{ij})_{i,j=0,\dots,n-1}$ where $c^k_{ij} = \hat{b}_i^T \cdot V(a_k \circ V^{-1}(\hat{b}_j))$

In [9]:
reg_rep = dict()

for k in range(n):
    c_k = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            c_k[i][j] = np.dot(b[i].transpose(), V(grp.op(a[k], Vinv(b[j]))))
    reg_rep[a[k]] = c_k

reg_rep

{'R0': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'R90': array([[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]),
 'R180': array([[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]]),
 'R270': array([[0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]])}

### Check Answer

Test to see if each matrix, computed for <b>reg_rep</b>, equals its counterpart in <b>z4_reg_rep</b>, defined in the Introduction section, above.

In [10]:
all([np.array_equal(reg_rep[elem], z4_reg_rep[elem]) for elem in grp.elements])

True

## Putting it All Together

Here is the desired implementation.

In [11]:
def regular_representation(grp):
    '''Given a group, this function returns three things: (1) A dictionary that maps each group
    element to its corresponding regular representation, (2) A function that maps a group element
    to its corresponding regular representation matrix, and (3) Another function that maps in the
    opposite direction, from regular represenation matrix to group element.'''
    
    a = grp.elements
    n = grp.order
    
    # Create a list of n nx1 orthogonal unit vectors
    id = np.eye(n, dtype=int)  # The nxn identity matrix
    b = [id[:,[i]] for i in range(n)]  # A list of the columns of the identity matrix
    
    # map group elements to vectors
    mapping = dict(zip(a, b))

    def V(elem):
        return mapping[elem]

    # Turn a column vector into a tuple, for use as a dict key
    def to_tuple(vec):
        return tuple(map(lambda x: x[0], list(vec)))
    
    # map vectors to group elements
    inv_mapping = {to_tuple(val): key for key, val in mapping.items()}
    
    def Vinv(vec):
        return inv_mapping[to_tuple(vec)]

    # Derive the n nxn matrices of the regular representation
    reg_rep = dict()
    for k in range(n):
        c_k = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                c_k[i][j] = np.dot(b[i].transpose(), V(grp.op(a[k], Vinv(b[j]))))
        reg_rep[a[k]] = c_k
        
    def to_reg_rep(elem):
        '''Given a group element, return the corresponding regular representation matrix.'''
        return reg_rep[elem]
    
    def array_2d_to_tuple(arr):
        '''Turn a 2D nd.array into a tuple of tuples, for use as a dictionary key'''
        return tuple(map(lambda x: tuple(x), np.ndarray.tolist(arr)))
    
    inv_mapping_2d = {array_2d_to_tuple(arr): key for key, arr in reg_rep.items()}
    
    def to_group_element(arr):
        '''Given a regular representation matrix, return the corresponding group element.'''
        return inv_mapping_2d[array_2d_to_tuple(arr)]
    
    def verify_regular_representation(reg_rep, grp):
        '''Verifies that rr(ai) x rr(aj) == rr(ai * aj), for all elements ai & aj of the group,
        where rr is the regular representation (maps group elements to reg reps), x is matrix
        multiplication, and * is the group operation.'''
        return all([np.array_equal(np.dot(to_reg_rep(a), to_reg_rep(b)), to_reg_rep(grp.op(a, b)))
                    for a in grp
                    for b in grp])
    
    if not verify_regular_representation(reg_rep, grp):
        raise AssertionError("Verification of Regular Represention failed")

    return reg_rep, to_reg_rep, to_group_element        

### Test/Example

A test and some printouts to see how it all works:

In [12]:
reg_rep, grp_to_rr, rr_to_grp = regular_representation(grp)

In [13]:
print(reg_rep)

{'R0': array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), 'R90': array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]]), 'R180': array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]]), 'R270': array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])}


In [14]:
for elem in grp:
    print(elem)
    rr = grp_to_rr(elem)
    print(rr)
    print(rr_to_grp(rr))
    print('')

R0
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
R0

R90
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
R90

R180
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
R180

R270
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]
R270



## More Tests / Examples

In [15]:
import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

ex = alg.Examples(alg_dir)

                           Example Algebras
----------------------------------------------------------------------
  17 example algebras are available.
  Use "Examples[INDEX]" to retrieve a specific example,
  where INDEX is the first number on each line below:
----------------------------------------------------------------------
0: A4 -- Alternating group on 4 letters (AKA Tetrahedral group)
1: D3 -- https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
2: D4 -- Dihedral group on four vertices
3: Pinter29 -- Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
4: RPS -- Rock, Paper, Scissors Magma
5: S3 -- Symmetric group on 3 letters
6: S3X -- Another version of the symmetric group on 3 letters
7: V4 -- Klein-4 group
8: Z4 -- Cyclic group of order 4
9: F4 -- Field with 4 elements (from Wikipedia)
10: mag_id -- Magma with Identity
11: Example 1.4.1 -- See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
12: Ex6 -- Example 6: http://www-groups.m

### $S_3$ the symmetric group on 3 letters

In [16]:
s3 = ex[5]
s3.about()


** Group **
Name: S3
Instance ID: 4636277264
Description: Symmetric group on 3 letters
Order: 6
Identity: e
Commutative? No
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       r     r^2       3
      2     r^2       r       3
      3       f       f       2
      4      fr      fr       2
      5      rf      rf       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5],
 [1, 2, 0, 5, 3, 4],
 [2, 0, 1, 4, 5, 3],
 [3, 4, 5, 0, 1, 2],
 [4, 5, 3, 2, 0, 1],
 [5, 3, 4, 1, 2, 0]]


In [17]:
s3_reg_rep, s3_to_rr, rr_to_s3 = regular_representation(s3)

In [18]:
s3_reg_rep

{'e': array([[1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.]]),
 'r': array([[0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.]]),
 'r^2': array([[0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.]]),
 'f': array([[0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.]]),
 'fr': array([[0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [1.,

In [19]:
for elem in s3:
    print(elem)
    rr = s3_to_rr(elem)
    print(rr)
    print(rr_to_s3(rr))
    print('')

e
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
e

r
[[0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]]
r

r^2
[[0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]
r^2

f
[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
f

fr
[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]
fr

rf
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
rf



### $v_4$ the Klein-4 group

In [20]:
v4 = ex[7]
v4.about()


** Group **
Name: V4
Instance ID: 4636142864
Description: Klein-4 group
Order: 4
Identity: e
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       h       h       2
      2       v       v       2
      3       r       r       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]


In [21]:
v4_reg_rep, v4_to_rr, rr_to_v4 = regular_representation(v4)

In [22]:
v4_reg_rep

{'e': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'h': array([[0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.]]),
 'v': array([[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]]),
 'r': array([[0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]])}

In [23]:
for elem in v4:
    print(elem)
    rr = v4_to_rr(elem)
    print(rr)
    print(rr_to_v4(rr))
    print('')

e
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
e

h
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
h

v
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
v

r
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
r



## Monoids Work as Well

In [24]:
m4 = alg.generate_commutative_monoid(4)
m4.about()


** Monoid **
Name: M4
Instance ID: 4644826704
Description: Autogenerated commutative Monoid of order 4
Order: 4
Identity: a1
Associative? Yes
Commutative? Yes
Cyclic?: No
Elements: ['a0', 'a1', 'a2', 'a3']
Has Inverses? No
Cayley Table (showing indices):
[[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 0, 2], [0, 3, 2, 1]]


In [25]:
m4_reg_rep, m4_to_rr, rr_to_m4 = regular_representation(m4)

In [26]:
m4_reg_rep

{'a0': array([[1., 1., 1., 1.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 'a1': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'a2': array([[1., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 1., 0., 1.],
        [0., 0., 0., 0.]]),
 'a3': array([[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.]])}

In [27]:
m5 = alg.generate_commutative_monoid(5)
m5.about()


** Monoid **
Name: M5
Instance ID: 4644833040
Description: Autogenerated commutative Monoid of order 5
Order: 5
Identity: a1
Associative? Yes
Commutative? Yes
Cyclic?: No
Elements: ['a0', 'a1', 'a2', 'a3', 'a4']
Has Inverses? No
Cayley Table (showing indices):
[[0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4],
 [0, 2, 4, 1, 3],
 [0, 3, 1, 4, 2],
 [0, 4, 3, 2, 1]]


In [28]:
m5_reg_rep, m5_to_rr, rr_to_m5 = regular_representation(m5)

In [29]:
m5_reg_rep

{'a0': array([[1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 'a1': array([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]]),
 'a2': array([[1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.]]),
 'a3': array([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.]]),
 'a4': array([[1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])}